In [ ]:
!pip install streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 28.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 54.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 8.0 MB/s eta 0:00:00


In [ ]:
# job_recommendation_app.py
%%writefile myapp.py
import streamlit as st
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics.pairwise import cosine_similarity
import re
import warnings

# Ignore all warnings
warnings.filterwarnings("ignore")
# Sample job listings data
job_data = {
    'job_id': [1, 2, 3, 4, 5],
    'job_title': ['Data Scientist', 'Software Engineer', 'Data Analyst', 'Product Manager', 'DevOps Engineer'],
    'job_description': [
        'Analyze data and build predictive models using machine learning algorithms.',
        'Develop software applications and work in a team.',
        'Analyze and interpret data to assist decision-making.',
        'Oversee product development and manage teams.',
        'Build and maintain infrastructure for deployment of applications.'
    ],
    'skills': [
        'Python, Machine Learning, Data Analysis',
        'Java, Python, Software Development',
        'Data Analysis, Python, SQL',
        'Product Management, Agile, Leadership',
        'DevOps, Cloud, Python, Docker'
    ]
}

jobs_df = pd.DataFrame(job_data)

# Preprocessing function to clean text
def preprocess_text(text):
    return re.sub(r'[^a-zA-Z\s]', '', text).lower()

# Vectorize job descriptions and skills
tfidf_vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf_vectorizer.fit_transform(jobs_df['job_description'].apply(preprocess_text))
skills_vectorizer = TfidfVectorizer(stop_words='english')
skills_matrix = skills_vectorizer.fit_transform(jobs_df['skills'].apply(preprocess_text))

# Combine job features
job_features = np.hstack([tfidf_matrix.toarray(), skills_matrix.toarray()])

# Content-based filtering function
def recommend_jobs(user_skills, top_n=3):
    user_skills_cleaned = preprocess_text(user_skills)
    user_skills_tfidf = skills_vectorizer.transform([user_skills_cleaned])
    similarity_scores = cosine_similarity(user_skills_tfidf, skills_matrix)
    similar_jobs_indices = similarity_scores[0].argsort()[-top_n:][::-1]
    recommended_jobs = jobs_df.iloc[similar_jobs_indices]
    return recommended_jobs

# Collaborative filtering setup
interaction_data = {
    'user_id': [1, 1, 2, 2, 3],
    'job_id': [1, 3, 2, 4, 5],
    'interaction': [1, 1, 1, 1, 1]
}

interaction_df = pd.DataFrame(interaction_data)
user_job_matrix = pd.pivot_table(interaction_df, index='user_id', columns='job_id', values='interaction', fill_value=0)
knn = NearestNeighbors(metric='cosine', algorithm='brute')
knn.fit(user_job_matrix)

# Collaborative filtering recommendation function
def recommend_jobs_collaborative(user_id, top_n=3):
    distances, indices = knn.kneighbors(user_job_matrix.loc[user_id].values.reshape(1, -1), n_neighbors=2)
    similar_users = user_job_matrix.iloc[indices[0]].sum(axis=0).sort_values(ascending=False)
    recommended_jobs = similar_users[~similar_users.index.isin(user_job_matrix.columns[user_job_matrix.loc[user_id] > 0])].head(top_n)
    return jobs_df[jobs_df['job_id'].isin(recommended_jobs.index)]

def app():
# Streamlit UI
  st.title("Job Recommendation System")

  st.sidebar.header("User Profile")
  user_skills = st.sidebar.text_input("Enter your skills (e.g., Python, Machine Learning):")
  user_id = st.sidebar.number_input("Enter your User ID for collaborative filtering:", min_value=1, step=1)

  st.write("## Recommended Jobs")

  if st.button("Recommend Based on Skills (Content-Based)"):
      if user_skills:
          recommendations = recommend_jobs(user_skills)
          st.write("**Content-Based Recommendations:**")
          for _, row in recommendations.iterrows():
              st.write(f"**Job Title:** {row['job_title']}")
              st.write(f"**Skills Required:** {row['skills']}")
              st.write("---")
      else:
          st.warning("Please enter your skills to get content-based recommendations.")

  if st.button("Recommend Based on Similar Users (Collaborative)"):
      try:
          recommendations_collab = recommend_jobs_collaborative(user_id)
          st.write("**Collaborative Filtering Recommendations:**")
          for _, row in recommendations_collab.iterrows():
              st.write(f"**Job Title:** {row['job_title']}")
              st.write(f"**Skills Required:** {row['skills']}")
              st.write("---")
      except KeyError:
          st.warning("User ID not found in interaction data. Try another User ID.")
app()

Writing myapp.py


In [ ]:
print("############")
print("IPv4")
!curl https://ipv4.icanhazip.com/
print("############")

############
IPv4
34.106.150.129
############


In [ ]:
!streamlit run myapp.py & npx localtunnel --port 8501



⠙⠹⠸
  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.106.150.129:8501

⠼⠴Need to install the following packages:
localtunnel@2.0.2
Ok to proceed? (y) y

⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋your url is: https://deep-months-shake.loca.lt
/root/.npm/_npx/75ac80b86e83d4a2/node_modules/localtunnel/bin/lt.js:81
    throw err;
    ^

Error: connection refused: localtunnel.me:31725 (check your firewall settings)
    at Socket.<anonymous> (/root/.npm/_npx/75ac80b86e83d4a2/node_modules/localtunnel/lib/TunnelCluster.js:52:11)
    at Socket.emit (node:events:517:28)
    at emitErrorNT (node:internal/streams/destroy:151:8)
    at emitErrorCloseNT (node:internal/streams/destroy:116:3)
    at process.processTicksAndRejections (node:internal/process/task_queues:82:21)

Node.js v18.20.5
⠙

In [ ]:
y
